In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from dateutil import parser
from datetime import datetime

In [3]:
df1 = pd.read_excel('O2D_delay_data.xlsx', 'logistics')
df2 = pd.read_excel('O2D_delay_data.xlsx', 'po_level')
df2 = df2[df2['order_status'].str.lower() == 'completed']

In [5]:
df2_selected_cols = df2[['buyer_name', 'seller_name', 'seller_po_number',
       'transporter_type', 'order_type',
       'order_completion_percentage', 'o2d_', 'O2D_Bucket']]
df = df1.merge(df2_selected_cols, left_on='po_number', right_on = 'seller_po_number', how = 'left')

# df['vehicle_unloaded_ts'] = pd.to_datetime(df['vehicle_unloaded_ts'], format='%m/%d/%y %H:%M:%S')
df = df.sort_values(by=['po_number', 'vehicle_unloaded_ts'])

# ## removing all the po which has NA in dispatched_qty
# dispatch_qty_na_po = set(df[df['dispatched_qty'].isna()]['po_number'].unique())
# print('Shape of df before filtering the po with NAN dispatch_qty')
# print(df.shape)
# df = df[~df['po_number'].isin(dispatch_qty_na_po)]
# print('Shape of df after filtering the po with NAN dispatch_qty')
print(df.shape)

## Renaming the category
df.loc[(df['category'] == 'RMX - Steel'), 'category'] = 'RMC - Steel'
df.loc[(df['transporter_type'] == 'Bizongo Non O2D'), 'transporter_type'] = 'Bizongo'

## Ranking the order of the transport
df['order'] = df.groupby('po_number')[['vehicle_reached_delivery_location_ts']].rank(method='first')
df = df[~df['po_number'].isin(['PO/25/MH/498','PO/25/MH/564', 'PO/25/MH/568'])]
df = df[~df['po_qty'].isna()]

## Route
df['route'] = df['origin'].str.lower() + '-' + df['destination'].str.lower()
df['lane_order'] = df.groupby('route')[['po_ts']].rank(method='min')


### Month wise
df['po_ts'] = df['po_ts'].astype(str)
def safe_parse(date_str):
    try:
        return parser.parse(date_str)
    except (ValueError, TypeError):
        return pd.NaT

df['po_ts'] = df['po_ts'].apply(safe_parse)
df['Month'] = df['po_ts'].dt.strftime('%B')


### Total rounds 

po_count = df['po_number'].value_counts().reset_index()
po_count.rename(columns = {'count' : 'total_rounds'}, inplace = True)
df = df.merge(po_count, on = 'po_number')

print(df.shape)

(163, 41)
(158, 46)


In [6]:
from datetime import datetime
import pandas as pd
import numpy as np

def convert_date_format(date_str):
    if pd.isna(date_str):
        return np.nan
    
    if isinstance(date_str, pd.Timestamp):
        return date_str.strftime("%m/%d/%Y %H:%M:%S")
    
    if isinstance(date_str, str):
        # Remove AM/PM if present
        date_str = date_str.replace(" AM", "").replace(" PM", "")
        
        # Remove any extra spaces before the time part
        date_str = date_str.strip()
        if ' :' in date_str:
            date_str = date_str.replace(' :', ' ')
        
        # Add seconds if not present in the input string
        if len(date_str.split(':')) == 2:
            date_str += ":00"
        
        # Try different possible input formats for string inputs
        possible_formats = [
            "%m/%d/%y %H:%M:%S",
            "%m/%d/%Y %H:%M:%S",
            "%-m/%-d/%y %H:%M:%S",   # Single-digit month and day
            "%-m/%-d/%Y %H:%M:%S",   # Single-digit month and day
        ]
        
        for fmt in possible_formats:
            try:
                dt = datetime.strptime(date_str, fmt)
                return dt.strftime("%m/%d/%Y %H:%M:%S")
            except ValueError:
                pass
        
    raise ValueError(f"Date format not recognized for '{date_str}'")



###############    Unloading Data     #############
df['vehicle_reached_delivery_location_ts'] = df['vehicle_reached_delivery_location_ts'].apply(lambda x: convert_date_format(x))
df['payment_confirmation_received_by_ops'] = df['payment_confirmation_received_by_ops'].apply(lambda x: convert_date_format(x))
df['vehicle_unloaded_ts'] = df['vehicle_unloaded_ts'].apply(lambda x: convert_date_format(x))

# Convert the date columns back to datetime
df['vehicle_reached_delivery_location_ts'] = pd.to_datetime(df['vehicle_reached_delivery_location_ts'])
df['payment_confirmation_received_by_ops'] = pd.to_datetime(df['payment_confirmation_received_by_ops'])
df['vehicle_unloaded_ts'] = pd.to_datetime(df['vehicle_unloaded_ts'])


df['unloading_time_total'] = (df['vehicle_unloaded_ts'] - df['vehicle_reached_delivery_location_ts']).dt.total_seconds() / (3600 * 24)
df['reached_to_payment_tat'] = (df['payment_confirmation_received_by_ops'] - df['vehicle_reached_delivery_location_ts']).dt.total_seconds() / (3600 * 24)
df['payment_to_unloading_tat'] = (df['vehicle_unloaded_ts'] - df['payment_confirmation_received_by_ops']).dt.total_seconds() / (3600 * 24)



############    Seller Level   ##########

df['vehicle_dispatched_ts'] = df['vehicle_dispatched_ts'].apply(lambda x: convert_date_format(x))
df['vehicle_reported_ts'] = df['vehicle_reported_ts'].apply(lambda x: convert_date_format(x))
df['vehicle_loaded_ts'] = df['vehicle_loaded_ts'].apply(lambda x: convert_date_format(x))

# Convert the date columns back to datetime
df['vehicle_dispatched_ts'] = pd.to_datetime(df['vehicle_dispatched_ts'])
df['vehicle_reported_ts'] = pd.to_datetime(df['vehicle_reported_ts'])
df['vehicle_loaded_ts'] = pd.to_datetime(df['vehicle_loaded_ts'])


df['seller_total_time'] = (df['vehicle_dispatched_ts'] - df['vehicle_reported_ts']).dt.total_seconds() / (3600 * 24)
df['vehicle_reported_to_load_tat'] = (df['vehicle_loaded_ts'] - df['vehicle_reported_ts']).dt.total_seconds() / (3600 * 24)
df['vehicle_loaded_to_dispatch_tat'] = (df['vehicle_dispatched_ts'] - df['vehicle_loaded_ts']).dt.total_seconds() / (3600 * 24)

###########   Transporter Level    ########33

df['nextbuy_vehicle_indent_ts'] = df['nextbuy_vehicle_indent_ts'].apply(lambda x: convert_date_format(x))
# df['vehicle_placement_ts'] = df['vehicle_placement_ts'].apply(lambda x: convert_date_format(x))
df['vehicle_unloaded_ts'] = df['vehicle_unloaded_ts'].apply(lambda x: convert_date_format(x))
df['vehicle_reported_ts'] = df['vehicle_reported_ts'].apply(lambda x: convert_date_format(x))
df['vehicle_dispatched_ts'] = df['vehicle_dispatched_ts'].apply(lambda x: convert_date_format(x))
df['vehicle_reached_delivery_location_ts'] = df['vehicle_reached_delivery_location_ts'].apply(lambda x: convert_date_format(x))


df['transporter_time'] = df['vehicle_reported_actual_vs_expected']
df['transporter_indent_to_accept_tat'] = (pd.to_datetime(df['transporter_order_indent_acceptance_ts']) - pd.to_datetime(df['transporter_order_indent_ts'])).dt.total_seconds() / (3600 * 24)
df['transporter_accept_to_placement_tat'] = (pd.to_datetime(df['vehicle_reported_ts']) - pd.to_datetime(df['transporter_order_indent_acceptance_ts'])).dt.total_seconds() / (3600 * 24)




#############     Internal      ################
df['vehicle_dispatched_ts'] = pd.to_datetime(df['vehicle_dispatched_ts'], errors='coerce')
df['nextbuy_vehicle_indent_ts'] = pd.to_datetime(df['nextbuy_vehicle_indent_ts'].apply(convert_date_format))
df['transporter_order_indent_acceptance_ts'] = pd.to_datetime(df['transporter_order_indent_acceptance_ts'].apply(convert_date_format))


df['po_ts'] = pd.to_datetime(df['po_ts'], errors='coerce')
df['expected_date_of_vehicle_reporting'] = pd.to_datetime(df['expected_date_of_vehicle_reporting'], errors='coerce')
df['vehicle_indent_to_accept_tat'] = (df['transporter_order_indent_acceptance_ts'] - df['nextbuy_vehicle_indent_ts']).dt.total_seconds() / (3600 * 24)


df['po_ts_to_expected_reporting'] = (df['expected_date_of_vehicle_reporting'] - df['po_ts']).dt.total_seconds() / (3600 * 24)



route = df[['route']]
df = pd.get_dummies(df, columns=['route'], prefix='route')
df = df.replace({True: 1, False: 0})
df = pd.concat([df, route], axis=1)

# df[df['delay_in_unloading'] >= 1].head()


df['steel'] = df['category'].apply(lambda x : 1 if x == 'RMC - Steel' else 0)
df['al'] = df['steel'].apply(lambda x : 1 if x == 0 else 0)

In [ ]:
df

## Input : 
1. Category
2. po_qty
3. seller name / buyer_name
4. origin / destination
5. transporter type (not required)
6. order type (not required for now)

Output : 
o2d value


prove that category is important than po_Qty. 
In short, prove how the fitering order is correct
.

In [30]:
"""
Objective : To have the data ready for O2D prediction.
Input : 
1. Po level data (xlsx format)
2. Logistics data (xlsx format)

Output : 
A dictionary with all the values.
"""

class value_calculator:
    def __init__(self, data):
        self.df = data

    def create_dict(self, df):
        value_dict = {}

                        #########################         category         ##########################
        
        value_dict = {'category' : {'aluminium' : {'range' : {}, 'qty_bin' : {}}, 
                                  'steel' : {'range' : {}, 'qty_bin' : {}}},
                                 'buyer' : {}, 'seller' : {}, 'route' : {}}
        ## aluminium
        aluminium_df = df[df['category'].str.contains('Aluminium', case=False, na=False)]
        al_3_std = self.find_std(aluminium_df, 3)
        al_mean = al_3_std['o2d_'].mean()
        al_max = al_3_std['o2d_'].max()
        al_min = al_3_std['o2d_'].min()
        value_dict['category']['aluminium']['range'] = [al_mean, al_min, al_max]
        

                  ######################       PO QTY     #######################
        bins = [0, 20, 40, 1000]
        labels = ['0-20', '21-40', '41-60']
        al_3_std['qty_bin'] = pd.cut(al_3_std['po_qty'], bins=bins, labels=labels, right=True, include_lowest=True)
        qty_wise_o2d = al_3_std.groupby('qty_bin')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        value_dict['category']['aluminium']['qty_bin'] = qty_wise_o2d.set_index('qty_bin').T.to_dict()

            ## Steel
        steel_df = df[df['category'].str.contains('Steel', case=False, na=False)]
        st_3_std = self.find_std(steel_df, 3)
        st_mean = st_3_std['o2d_'].mean()
        st_max = st_3_std['o2d_'].max()
        st_min = st_3_std['o2d_'].min()
        value_dict['category']['steel']['range'] = [st_mean, st_min, st_max]  
               
        bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 1000]
        labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']    
        st_3_std['qty_bin'] = pd.cut(st_3_std['po_qty'], bins=bins, labels=labels, right=False, include_lowest=True)
        qty_wise_o2d = st_3_std.groupby('qty_bin')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        value_dict['category']['steel']['qty_bin'] = qty_wise_o2d.set_index('qty_bin').T.to_dict()

        ##############################     Buyer / Seller    ################################
        df3 = self.find_std(df, 3)
        
        ### Buyer
        
        overall_stats = df3['o2d_'].agg(['mean', 'max', 'min']).to_frame().T
        overall_stats['buyer_name'] = 'other'

        correlations = df3.groupby('buyer_name')['o2d_'].apply(lambda x: x.corr(df3['o2d_']))
        filtered_buyers = correlations[(correlations > 0.4) | (correlations < -0.4)].index.tolist()
        temp_df = df3[df3['buyer_name'].isin(set(filtered_buyers))]
        quant_values = temp_df.groupby('buyer_name')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        
        quant_values = pd.concat([quant_values, overall_stats], ignore_index=True)
        value_dict['buyer'] = quant_values.set_index('buyer_name').T.to_dict()


        ### Seller

        correlations = df3.groupby('seller_name')['o2d_'].apply(lambda x: x.corr(df3['o2d_']))
        filtered_sellers = correlations[(correlations > 0.4) | (correlations < -0.4)].index.tolist()

        temp_df = df3[df3['seller_name'].isin(set(filtered_sellers))]
        quant_values = temp_df.groupby('buyer_name')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        
        overall_stats = df3['o2d_'].agg(['mean', 'max', 'min']).to_frame().T
        overall_stats['buyer_name'] = 'other'
        
        quant_values = pd.concat([quant_values, overall_stats], ignore_index=True)
        if 'seller_name' in quant_values.columns:
            value_dict['seller'] = quant_values.set_index('seller_name').T.to_dict()
        else:
            # Handle case where 'seller_name' is missing
            value_dict['seller'] = quant_values.to_dict()


    ##########################        Lane        ###########################

        df3['route'] = df3['origin'] + '-' + df3['destination']
        correlations = df3.groupby('route')['o2d_'].apply(lambda x: x.corr(df3['o2d_']))
        filtered_routes = correlations[(correlations > 0.4) | (correlations < -0.4)].index.tolist()

        temp_df = df3[df3['route'].isin(set(filtered_sellers))]
        quant_values = temp_df.groupby('route')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        
        overall_stats = df3['o2d_'].agg(['mean', 'max', 'min']).to_frame().T
        overall_stats['route'] = 'other'
        
        quant_values = pd.concat([quant_values, overall_stats], ignore_index=True)
        value_dict['route'] = quant_values.set_index('route').T.to_dict()       

        return df, value_dict

    def find_std(self, df, std):
        """
        Define the std.
        """
        mean = df['o2d_'].mean()
        std_dev = df['o2d_'].std()
        
        lower_bound = mean - std * std_dev
        upper_bound = mean + std * std_dev
        filtered_df = df[(df['o2d_'] >= lower_bound) & (df['o2d_'] <= upper_bound)]
        return filtered_df
    
    def preprocessing(self, df):

        df.rename(columns = { 'vehicle_level_o2d': 'o2d_'}, inplace = True)
        df.loc[(df['category'] == 'RMX - Steel'), 'category'] = 'RMC - Steel'
        df.loc[(df['transporter_type'] == 'Bizongo Non O2D'), 'transporter_type'] = 'Bizongo'
        df['order'] = df.groupby('po_number')[['vehicle_reached_delivery_location_ts']].rank(method='first')
        df = df[~df['po_number'].isin(['PO/25/MH/498','PO/25/MH/564', 'PO/25/MH/568'])]
        df = df[~df['po_qty'].isna()]
        
        ## Route
        # df['route'] = df['origin'].str.lower() + '-' + df['destination'].str.lower()
        df['lane_order'] = df.groupby('route')[['po_ts']].rank(method='min')
        
        
        ### Month wise
        df['po_ts'] = df['po_ts'].astype(str)
        def safe_parse(date_str):
            try:
                return parser.parse(date_str)
            except (ValueError, TypeError):
                return pd.NaT
        
        df['po_ts'] = df['po_ts'].apply(safe_parse)
        df['Month'] = df['po_ts'].dt.strftime('%B')
        
        
        ### Total rounds 
        
        po_count = df['po_number'].value_counts().reset_index()
        po_count.rename(columns = {'count' : 'total_rounds'}, inplace = True)
        df = df.merge(po_count, on = 'po_number')
                
        ## removing april data
        df = df[df['Month'] != 'April']
        return df


    def main(self):
        self.df = self.preprocessing(self.df)
        final_used_df, value_dict = self.create_dict(self.df)
        return value_dict
        




class o2d_predictor:
    def __init__(self, category, po_qty, buyer_name, seller_name, origin, destination, df):
        self.category = category.lower()
        self.po_qty = int(po_qty)
        self.buyer_name = buyer_name
        self.seller_name = seller_name
        self.origin = origin
        self.destination = destination
        self.value_dict = self.value_calculator(df)

    def o2d_calculator(self):

        ### can add weights based on featue importance, etc.

        ## po_qty

        po_qty = 0
        for bin in self.value_dict[self.category]['qty_bin']:
            lower, upper = bin.split('-')
            if int(lower.strip()) <= self.po_qty <= int(lower.strip()):
                po_qty = self.value_dict[self.category]['qty_bin'][bin]
                break

        ## Buyer
        if self.buyer_name not in self.value_dict['buyer']:
            buyer_val = self.value_dict['buyer']['other']
        else:
            buyer_val = self.value_dict['buyer']['other']

        ## Seller
        if self.seller_name not in self.value_dict['seller']:
            seller_val = self.value_dict['seller']['other']
        else:
            seller_val = self.value_dict['seller']['other']   


        ### Lane
        lane_name = self.origin.lower() + '-' + self.destination.lower()
        if lane_name not in self.value_dict['lane']:
            lane_val = self.value_dict['lane']['other']
        else:
            lane_val = self.value_dict['lane']['other']   
        
        
        return ((self.value_dict[self.category][0] + buyer_val + seller_val + lane_val)/4).round(2)


if __name__ == "__main__":

    df1 = pd.read_excel('O2D_delay_data.xlsx', 'logistics')
    df2 = pd.read_excel('O2D_delay_data.xlsx', 'po_level')
    df2 = df2[df2['order_status'].str.lower() == 'completed']
    df2 = df2[['buyer_name', 'seller_name', 
           'transporter_type', 'order_type','seller_po_number',
           'order_completion_percentage']]
    
    df = df1.merge(df2, left_on='po_number', right_on = 'seller_po_number', how = 'left')
    df = df.sort_values(by=['po_number', 'vehicle_unloaded_ts'])

    o2d_prediction = o2d_predictor('aluminium', 30, 'HANNU STEEL PRIVATE LIMITED', 	'SUMANGAL ISPAT PVT LTD', 'raipur', 'jaipur', df)
    print(o2d_prediction.o2d_calculator())


AttributeError: 'o2d_predictor' object has no attribute 'value_calculator'

In [24]:
"""
Objective : To have the data ready for O2D prediction.
Input : 
1. Po level data (xlsx format)
2. Logistics data (xlsx format)

Output : 
A dictionary with all the values.
"""

class value_calculator:
    def __init__(self, data):
        self.df = data

    def create_dict(self, df):
        value_dict = {}

                        #########################         category         ##########################
        
        value_dict = {'category' : {'aluminium' : {'range' : {}, 'qty_bin' : {}}, 
                                  'steel' : {'range' : {}, 'qty_bin' : {}}},
                                 'buyer' : {}, 'seller' : {}, 'route' : {}}
        ## aluminium
        aluminium_df = df[df['category'].str.contains('Aluminium', case=False, na=False)]
        al_3_std = self.find_std(aluminium_df, 3)
        al_mean = al_3_std['o2d_'].mean()
        al_max = al_3_std['o2d_'].max()
        al_min = al_3_std['o2d_'].min()
        value_dict['category']['aluminium']['range'] = [al_mean, al_min, al_max]
        

                  ######################       PO QTY     #######################
        bins = [0, 20, 40, 1000]
        labels = ['0-20', '21-40', '41-60']
        al_3_std['qty_bin'] = pd.cut(al_3_std['po_qty'], bins=bins, labels=labels, right=True, include_lowest=True)
        qty_wise_o2d = al_3_std.groupby('qty_bin')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        value_dict['category']['aluminium']['qty_bin'] = qty_wise_o2d.set_index('qty_bin').T.to_dict()

            ## Steel
        steel_df = df[df['category'].str.contains('Steel', case=False, na=False)]
        st_3_std = self.find_std(steel_df, 3)
        st_mean = st_3_std['o2d_'].mean()
        st_max = st_3_std['o2d_'].max()
        st_min = st_3_std['o2d_'].min()
        value_dict['category']['steel']['range'] = [st_mean, st_min, st_max]  
               
        bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 1000]
        labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']    
        st_3_std['qty_bin'] = pd.cut(st_3_std['po_qty'], bins=bins, labels=labels, right=False, include_lowest=True)
        qty_wise_o2d = st_3_std.groupby('qty_bin')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        value_dict['category']['steel']['qty_bin'] = qty_wise_o2d.set_index('qty_bin').T.to_dict()

        ##############################     Buyer / Seller    ################################
        df3 = self.find_std(df, 3)
        
        ### Buyer
        
        overall_stats = df3['o2d_'].agg(['mean', 'max', 'min']).to_frame().T
        overall_stats['buyer_name'] = 'other'

        correlations = df3.groupby('buyer_name')['o2d_'].apply(lambda x: x.corr(df3['o2d_']))
        filtered_buyers = correlations[(correlations > 0.4) | (correlations < -0.4)].index.tolist()
        temp_df = df3[df3['buyer_name'].isin(set(filtered_buyers))]
        quant_values = temp_df.groupby('buyer_name')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        
        quant_values = pd.concat([quant_values, overall_stats], ignore_index=True)
        value_dict['buyer'] = quant_values.set_index('buyer_name').T.to_dict()


        ### Seller

        correlations = df3.groupby('seller_name')['o2d_'].apply(lambda x: x.corr(df3['o2d_']))
        filtered_sellers = correlations[(correlations > 0.4) | (correlations < -0.4)].index.tolist()

        temp_df = df3[df3['seller_name'].isin(set(filtered_sellers))]
        quant_values = temp_df.groupby('buyer_name')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        
        overall_stats = df3['o2d_'].agg(['mean', 'max', 'min']).to_frame().T
        overall_stats['buyer_name'] = 'other'
        
        quant_values = pd.concat([quant_values, overall_stats], ignore_index=True)
        if 'seller_name' in quant_values.columns:
            value_dict['seller'] = quant_values.set_index('seller_name').T.to_dict()
        else:
            # Handle case where 'seller_name' is missing
            value_dict['seller'] = quant_values.to_dict()


    ##########################        Lane        ###########################

        df3['route'] = df3['origin'] + '-' + df3['destination']
        correlations = df3.groupby('route')['o2d_'].apply(lambda x: x.corr(df3['o2d_']))
        filtered_routes = correlations[(correlations > 0.4) | (correlations < -0.4)].index.tolist()

        temp_df = df3[df3['route'].isin(set(filtered_sellers))]
        quant_values = temp_df.groupby('route')['o2d_'].agg(['mean', 'max', 'min']).reset_index()
        
        overall_stats = df3['o2d_'].agg(['mean', 'max', 'min']).to_frame().T
        overall_stats['route'] = 'other'
        
        quant_values = pd.concat([quant_values, overall_stats], ignore_index=True)
        value_dict['route'] = quant_values.set_index('route').T.to_dict()       

        return df, value_dict

    def find_std(self, df, std):
        """
        Define the std.
        """
        mean = df['o2d_'].mean()
        std_dev = df['o2d_'].std()
        
        lower_bound = mean - std * std_dev
        upper_bound = mean + std * std_dev
        filtered_df = df[(df['o2d_'] >= lower_bound) & (df['o2d_'] <= upper_bound)]
        return filtered_df
    
    def preprocessing(self, df):

        df.rename(columns = { 'vehicle_level_o2d': 'o2d_'}, inplace = True)
        df.loc[(df['category'] == 'RMX - Steel'), 'category'] = 'RMC - Steel'
        df.loc[(df['transporter_type'] == 'Bizongo Non O2D'), 'transporter_type'] = 'Bizongo'
        df['order'] = df.groupby('po_number')[['vehicle_reached_delivery_location_ts']].rank(method='first')
        df = df[~df['po_number'].isin(['PO/25/MH/498','PO/25/MH/564', 'PO/25/MH/568'])]
        df = df[~df['po_qty'].isna()]
        
        ## Route
        # df['route'] = df['origin'].str.lower() + '-' + df['destination'].str.lower()
        df['lane_order'] = df.groupby('route')[['po_ts']].rank(method='min')
        
        
        ### Month wise
        df['po_ts'] = df['po_ts'].astype(str)
        def safe_parse(date_str):
            try:
                return parser.parse(date_str)
            except (ValueError, TypeError):
                return pd.NaT
        
        df['po_ts'] = df['po_ts'].apply(safe_parse)
        df['Month'] = df['po_ts'].dt.strftime('%B')
        
        
        ### Total rounds 
        
        po_count = df['po_number'].value_counts().reset_index()
        po_count.rename(columns = {'count' : 'total_rounds'}, inplace = True)
        df = df.merge(po_count, on = 'po_number')
                
        ## removing april data
        df = df[df['Month'] != 'April']
        return df


    def main(self):
        self.df = self.preprocessing(self.df)
        final_used_df, value_dict = self.create_dict(self.df)
        return value_dict
        


In [13]:
df1 = pd.read_excel('../O2D_delay_data.xlsx', 'logistics')
df2 = pd.read_excel('../O2D_delay_data.xlsx', 'po_level')
df2 = df2[df2['order_status'].str.lower() == 'completed']
df2 = df2[['buyer_name', 'seller_name', 
       'transporter_type', 'order_type',
       'order_completion_percentage']]

df = df1.merge(df2_selected_cols, left_on='po_number', right_on = 'seller_po_number', how = 'left')
df = df.sort_values(by=['po_number', 'vehicle_unloaded_ts'])

In [14]:
df.head()

,sno,date,po_number,po_qty,seller_po_creation_date,category,dispatched_qty,origin,destination,dispatch_week,po_ts,nextbuy_order_indent_ts,nextbuy_vehicle_indent_ts,transporter_order_indent_ts,transporter_order_indent_acceptance_ts,vehicle_placement_ts,expected_date_of_vehicle_reporting,vehicle_reported_ts,vehicle_loaded_ts,vehicle_dispatched_ts,vehicle_reached_delivery_location_ts,payment_confirmation_received_by_ops,vehicle_unloaded_ts,nextbuy_order_indent_tat,nextbuy_vehicle_indent_tat,vehicle_placement_tat,vehicle_reported_tat,vehicle_dispatch_tat,vehicle_reported_actual_vs_expected,transporter_order_tat,dispatch_to_delivery_tat,unloading_tat,vehicle_level_o2d,buyer_name,seller_name,seller_po_number,transporter_type,order_type,order_completion_percentage,o2d_,O2D_Bucket
145,152,2024-07-04,PO/25/CG/18,30.0,2024-07-04,RMC - Aluminium,NaN,Raipur,Medak,NaN,07/04/2024 8:31:02,07/04/2024 11:31:02,07/04/2024 11:31:02,07/04/2024 11:31:02,07/04/2024 11:31:02,07/04/2024 11:31:02,07/04/2024 12:31:02,07/04/2024 13:45:00,07/05/2024 18:10:00,07/05/2024 18:20:00,NaN,NaN,NaN,0.125012,0.000012,0.000012,0.093044,1.190984,0.051354,0.000012,2.236111,0.000000,NaN,Sagar Asia Private Limited,JAGDAMBA STEELS,PO/25/CG/18,Bizongo,S2B,98.416667,1.000000,0-3 days
36,37,2024-04-16,PO/25/HA/111,600.0,2024-04-16,RMC - Steel,30.000000,Durgapur,Sikendarabad,April 4 Week,04/16/2024 17:54:35,4/17/2024 16:23:00,4/17/2024 16:23:00,4/18/2024 10:20:00,4/18/2024 10:50:00,04/24/2024 7:31:00,4/19/2024 23:59:59,04/24/2024 4:00:00,04/24/2024 14:25:00,04/24/24 22:00:00,04/26/24 11:00:00,04/27/24 14:14:00,04/27/24 16:00:00,0.936412,0.000012,6.630567,-0.146516,0.750012,4.166667,0.747928,1.541678,1.208345,8.17043,HANNU STEEL PRIVATE LIMITED,SUMANGAL ISPAT PVT LTD,PO/25/HA/111,Bizongo,S2B,100.000000,14.759483,6+ days
4,5,2024-04-16,PO/25/HA/111,600.0,2024-04-16,RMC - Steel,25.129999,Durgapur,Sikendarabad,April 3 Week,04/16/2024 17:54:35,4/17/2024 16:23:00,4/17/2024 16:23:00,4/18/2024 10:20:00,4/18/2024 10:50:00,4/22/2024 11:00:00,4/19/2024 23:59:59,04/24/24 4:05:00,04/24/24 14:02:00,04/24/2024 22:15:00,4/26/2024 23:00:00,04/27/2024 16:15:00,04/28/2024 15:00:00,0.936412,0.000012,4.775706,1.711817,0.756956,4.170139,0.747928,2.031262,1.666678,8.18085,HANNU STEEL PRIVATE LIMITED,SUMANGAL ISPAT PVT LTD,PO/25/HA/111,Bizongo,S2B,100.000000,14.759483,6+ days
39,40,2024-04-16,PO/25/HA/111,600.0,2024-04-16,RMC - Steel,42.040001,Durgapur,Sikendarabad,April 4 Week,04/16/2024 17:54:35,4/17/2024 16:23:00,4/17/2024 16:23:00,4/18/2024 10:20:00,4/18/2024 10:50:00,04/29/2024 9:03:00,4/20/2024 23:59:59,04/29/24 5:00:00,04/29/24 17:05:00,04/29/24 21:00:00,5/2/2024 9:00:00,05/02/2024 11:00:00,05/02/2024 13:00:00,0.936412,0.000012,11.694456,-0.168738,0.666678,8.208333,0.747928,2.500012,0.166678,13.12880,HANNU STEEL PRIVATE LIMITED,SUMANGAL ISPAT PVT LTD,PO/25/HA/111,Bizongo,S2B,100.000000,14.759483,6+ days
38,39,2024-04-16,PO/25/HA/111,600.0,2024-04-16,RMC - Steel,41.520000,Durgapur,Sikendarabad,April 4 Week,04/16/2024 17:54:35,4/17/2024 16:23:00,4/17/2024 16:23:00,4/18/2024 10:20:00,4/18/2024 10:50:00,04/28/2024 22:12:00,4/20/2024 23:59:59,4/29/2024 5:00:00,04/29/24 16:29:00,04/29/2024 22:00:00,05/02/2024 9:28:00,05/02/2024 13:01:00,05/02/2024 13:30:24,0.936412,0.000012,11.242373,0.283345,0.708345,8.208333,0.747928,2.477789,0.168345,13.17040,HANNU STEEL PRIVATE LIMITED,SUMANGAL ISPAT PVT LTD,PO/25/HA/111,Bizongo,S2B,100.000000,14.759483,6+ days


In [31]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
